In [ ]:
import math
import random

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import mlx.core as mx
import scipy

**Q1. Murphy's Brier Decomposition.**  Suppose there are $n$ examples $(x_1, y_1), \ldots, (x_n, y_n)$ with binary labels $y_i \in \{0, 1\}$ and probability forecasts $p_1, \ldots, p_n \in [0, 1].$ Partition the index set $\{1, \ldots, n\}$ arbitrarily into disjoint groups $S_1, \ldots, S_B$ whose union is $\{1, \ldots n\}.$ For each bin $b$, define $n_b = |S_b|, w_b = n_b / n$, the empirical event rate $r_b = \frac{1}{n_b} \sum_{i \in S_b} y_i,$ and the average forecast $c_b = \frac{1}{n_b} \sum_{i \in S_b} p_i$. Let the overall event rate be $r = \frac{1}{n} \sum_{i = 1}^{n} y_i.$ Show the empirical identity
$$
\text{Brier score} = \frac{1}{n} \sum_{i=1}^{n} (y_i - p_i)^{2} = \underbrace{r (1 - r)}_{\text{uncertainty}} - \underbrace{\sum_{b = 1}^{B} w_b (r_b - r)^2}_{\text{resolution}} + \underbrace{\sum_{b = 1}^{B} w_b (c_b - r_b)^2}_{\text{reliability}}.
$$

Provide a short derivation (e.g., add-and-subtract $r_b$, expand, and average within bins). Conclude that replacing each $p_i$ by $r_b$ for $i \in S_b$ (bin-wise recalibration) reduces the Brier score by exactly the reliability term and leaves uncertainty and resolution unchanged.

In [3]:
def brier(dataset, probs):
    return (1 / len(dataset)) * sum((yi - pi) ** 2 for (xi, yi), pi in zip(dataset, probs))

# [1] -> 1
# ----
# [[1]]

# [2] -> 2
# ----
# [[1], [2]]
# [[1, 2]]

# [1, 2, 3]
# -----------------
# [[1], [2], [3]]
# [[1, 2], [3]]
# [[1, 3], [2]]
# [[1], [2, 3]]
# [[1, 2, 3]]

def bell(n):
    if n == 0:
        return 1
    else:
        return sum(math.comb(n - 1, k) * bell(k) for k in range(n))

# TODO: preallocate in this function
def partitions(xs):
    if len(xs) == 0:
        return [[]]

    assert(len(xs) > 0)

    xs = sorted(xs)
    N = len(xs)

    acc = []
    def backtrack(part, sofar):
        if len(xs) == 0:
            sofar.append(sorted(part))
            sorted_sofar = sorted(sofar)
            if sorted_sofar not in acc:
                acc.append(sorted_sofar) 
            sofar.pop()
        elif len(acc) < bell(N):
            n = len(xs)
            for i in range(n):
                # continue class 
                part.append(xs.pop(i))
                backtrack(part, sofar)
                xs.insert(i, part.pop())

                # break class 
                if len(part) > 0:
                    sofar.append(sorted(part))
                    x = xs.pop(i)
                    backtrack([x], sofar)
                    sofar.pop()
                    xs.insert(i, x)
    backtrack([], [])

    assert len(acc) == bell(N)

    return acc


n = sympy.symbols('n')
Nn = 6

xs = sympy.symbols(f'x:{Nn}')
ys = sympy.symbols(f'y:{Nn}')
ps = sympy.symbols(f'p:{Nn}')
x0, x1, x2, x3, x4, x5 = xs 
y0, y1, y2, y3, y4, y5 = ys 
p0, p1, p2, p3, p4, p5 = ps 

dataset = [(xi, yi) for (xi, yi) in zip(xs, ys)]
index_set = {i for i in range(Nn)}
binss = partitions(index_set)

bins_ix = 34 # chosen randomly
bins = binss[bins_ix]

ns = sympy.symbols(f'n:{len(bins)}') 
n0, n1, n2 = ns
Nn0 = len(bins[0])
Nn1 = len(bins[1])
Nn2 = len(bins[2])

ws = sympy.symbols(f'w:{len(ns)}')
w0, w1, w2 = ws
Nw0 = n0.subs(n0, Nn0)
Nw1 = n1.subs(n1, Nn1)
Nw2 = n2.subs(n2, Nn2)

rs = [(1 / ns[b]) * sum(ys[i] for i in range(Nn) if i in bins[b]) for b in range(len(bins))]
r0, r1, r2 = rs

r = sum(yi for yi in ys) / n
uncertainty = r * (1 - r)

brier_score = (1 / n) * sum((yi - pi)**2 for yi, pi in zip(ys, ps))

print(bins)
print(rs)

[[0, 1, 3], [2, 5], [4]]
[(y0 + y1 + y3)/n0, (y2 + y5)/n1, y4/n2]


In [16]:
x = sympy.IndexedBase('x', shape=(n,))
y = sympy.IndexedBase('y', shape=(n,))
p = sympy.IndexedBase('p', shape=(n,))

x.subs('n', 10)[11]

x[11]